In [1]:
# !pip install torch

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss

from tqdm.auto import tqdm
import copy

np.random.seed(12345)
torch.manual_seed(123)

# ========== Data ==========

In [3]:
data = pd.read_csv('dataset3.csv')

In [4]:
data

,x1,x2,y
0,0,4,A
1,2,2,B
2,2,4,A
3,1,4,A
4,-1,3,A
5,0,2,A
6,3,1,B
7,-3,1,C
8,3,3,B
9,1,-2,C


In [5]:
X = data[['x1', 'x2']].to_numpy()

In [6]:
y = data['y'].to_numpy()

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True)

In [8]:
le = LabelEncoder()
y_le_train = le.fit_transform(y_train)
y_le_val = le.transform(y_train)

# ========== Model ==========

In [9]:
if torch.cuda.is_available():
  device = 'cuda:0'
else:
  device = 'cpu'

In [10]:
X_train = torch.from_numpy(X_train).float().to(device)
y_le_train = torch.from_numpy(y_le_train).long().to(device)

X_val = torch.from_numpy(X_val).float().to(device)
y_le_val = torch.from_numpy(y_le_val).long().to(device)

In [11]:
layers = []

layers.append(nn.Linear(X.shape[1], 5))
layers.append(nn.ReLU())

layers.append(nn.Linear(5, 4))
layers.append(nn.ReLU())

layers.append(nn.Linear(4, 3))

model = nn.Sequential(*layers).to(device)

In [12]:
best_loss = np.inf

alphas = np.logspace(-3, 2, 100)

for alpha in tqdm(alphas):
    optimizer = torch.optim.SGD(model.parameters(), lr=1)
    loss_function = nn.CrossEntropyLoss()
    for i in tqdm(range(500)):
        model.train()
        optimizer.zero_grad()

        y_pred_softmax_train = model(X_train)

        l2_norm = 0
        for layer in model.children():
            if isinstance(layer, nn.Linear):
                for params in layer.parameters():
                    l2_norm += sum(p.pow(2.0).sum() for p in params)

        loss = loss_function(y_pred_softmax_train, y_le_train) + alpha*l2_norm
        loss.backward()

        optimizer.step()
        
    with torch.no_grad():
        model.eval()
        y_pred_softmax_val = torch.log_softmax(model(X_val), dim=1)
        
    if np.isnan(y_pred_softmax_val.min()):
        continue
        
    loss = log_loss(y_val, y_pred_softmax_val, labels=np.unique(y))
    
    if loss < best_loss:
        best_loss = loss
        best_model = copy.deepcopy(model)
        best_alpha = alpha

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [13]:
best_alpha

0.001

In [14]:
for name, param in best_model.named_parameters():
    print(name, param)
    print()

0.weight Parameter containing:
tensor([[-0.4413, -0.0386],
        [-0.6465,  0.6212],
        [-0.7190,  0.8051],
        [-1.0394, -1.1434],
        [-0.6446,  0.8014]], requires_grad=True)

0.bias Parameter containing:
tensor([ 0.0765,  0.0669, -0.1570,  1.1372, -0.1282], requires_grad=True)

2.weight Parameter containing:
tensor([[-0.0340, -0.1464,  0.0810, -0.0388,  0.0323],
        [ 0.3700,  0.0925,  0.0747,  1.5827, -0.0908],
        [ 0.2814, -0.0276,  0.1289,  1.0339, -0.1797],
        [-0.0114,  0.8768,  1.0304, -0.2084,  1.0065]], requires_grad=True)

2.bias Parameter containing:
tensor([-0.0349, -0.0020,  0.1269, -0.1716], requires_grad=True)

4.weight Parameter containing:
tensor([[ 0.0210, -0.4780, -0.7373,  1.2722],
        [ 0.1704, -0.7555, -0.0291, -1.1040],
        [ 0.0238,  1.3484,  0.7860, -0.1483]], requires_grad=True)

4.bias Parameter containing:
tensor([-1.2898,  2.9828, -1.5388], requires_grad=True)



# ========== Prediction ==========

In [15]:
X = pd.read_csv('incoming_data3.csv').to_numpy()

In [16]:
X = torch.from_numpy(X).float().to(device)

In [17]:
with torch.no_grad():
    best_model.eval()
    y_pred_softmax = torch.log_softmax(best_model(X), dim=1)
    _, y_pred_le = torch.max(y_pred_softmax, dim=1)
y_pred = le.inverse_transform(y_pred_le)

y_pred

array(['C', 'A', 'B', 'A', 'C', 'B'], dtype=object)